In [111]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [112]:
import math

import numpy as np
import matplotlib.pyplot as plt
from itertools import product
from util.plots import scatter_plot
from util.data import load_mnist, encode_data
from typing import List, Dict
from tqdm.notebook import tqdm
from algorithms import GridCells
from algorithms.SaccadeGenerator import SaccadeGenerator

<IPython.core.display.Javascript object>

### Algorithm

1. movement to update location layer
    - calculate new phase - input: displacement vector and old phase; applies translation by rotation and scaling + mod 1 --> outputs binary vector A_t,move^loc
    - if no movement is given, activate random location ???
2. predict sensory information from location layer output
    - predictions represented by pi_t^in = activity of distal dendritic segments - Eq2 in paper
        --> modularoty effect
3. calculate activity in sensory layer
    - all cells share same feedforward receptive fields --> pretrained CNN ouputs (128,) feature vector and each column is mapped to on feature
    - sensory features represented by sparse subset of mini-columns = W_t^in
    - if cell is predicted and in active mini-columns: activate cell and inhibit other cells in mini-column; else: all cells become active
    - - multiple cells can be active if predicted by location layer
    - formula uq: 6 for activation logic
4. update location layer based on sensory cues
    - location layer receives input from sensory layer (in form of W_t^in???)
    - calculate overlap EQ 3 & 4
    - repeat because new movement is received

Learning:
- strenghtening between connections of both layer
- when new object is learned: location representation is randomly initialised (each object has its own location space = multiple objects can be represented at the same time because their representation has low probability of overlap
- sensory layer: random cell in each column will be active
- form reciprocal connections on one of their dendritic segments (d') (EQ 5 & 6)
--> if synapse exist it is unaffected otherwise it is formed

### Setup Data and Saccade Generator

In [113]:
(x_train, y_train), (x_test, y_test) = load_mnist()
x_train, x_test = encode_data(x_train, x_test)

<IPython.core.display.Javascript object>

In [114]:
saccade_size = (10, 10)

<IPython.core.display.Javascript object>

In [117]:
saccade_generator = SaccadeGenerator(
    x=x_train,
    y=y_train,
    saccade_size=saccade_size,
    no_saccades=5,
)

<IPython.core.display.Javascript object>

### Setup GridCells - Sensory Location Layers

In [118]:
no_features = math.prod(saccade_size)

<IPython.core.display.Javascript object>

In [119]:
gc = GridCells(
    no_modules=32,
    grid_cell_module_dimension=(16, 16),
    orientation=60,
    scale=20,
    no_columns=no_features,
    no_cells_per_column=8,
    location_layer_threshold=6,
    seed=42,
)

<IPython.core.display.Javascript object>

### Learn Objects

In [120]:
for obj in saccade_generator:
    gc.learn_object(obj)

    break

ValueError: operands could not be broadcast together with shapes (65,2) (100,) 

<IPython.core.display.Javascript object>